<a href="https://colab.research.google.com/github/miniii222/0.start/blob/master/project_code/model/make_test_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google 드라이브를 마운트하려면 이 셀을 실행하세요.
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.listdir('./drive/My Drive/colab/recsys/data')
path = './drive/My Drive/colab/recsys/data/'

In [0]:
import pandas as pd
import numpy as np

In [0]:
#train = pd.read_csv(path+'train.csv')
test = pd.read_csv(path + 'test.csv')

In [5]:
print('test data에서 맞춰야되는 개수 : ',len(test[test.reference.isnull()]))
print('test data에서 clickout 개수 : ', len(test[test.action_type == 'clickout item']))

test data에서 맞춰야되는 개수 :  261731
test data에서 clickout 개수 :  528779


In [0]:
528779 - 261731

267048

In [0]:
print('test data에서의 click out 하는 session id 개수 : ',len(test[(test.action_type == 'clickout item')]['session_id'].unique()))
print('test data에서 맞춰야 되는 session id 개수 : ',len(test[test.reference.isnull()]['session_id'].unique()))

test data에서의 click out 하는 session id 개수 :  275679
test data에서 맞춰야 되는 session id 개수 :  261731


In [0]:
not_na = list(set(test[(test.action_type == 'clickout item')]['session_id'].unique()) - set(test[test.reference.isnull()]['session_id'].unique()))

In [12]:
#NA인 clickout
na_clickout = test[(test.action_type == 'clickout item') & (test.reference.isnull())]
print('전체 개수 : ',len(na_clickout))
print('session_id unique : ',len(na_clickout.session_id.unique()))

전체 개수 :  253573
session_id unique :  253573


In [13]:
na_clickout = na_clickout[['user_id','session_id','timestamp','step']]
na_clickout.reset_index(inplace = True)
na_clickout.head()

,index,user_id,session_id,timestamp,step
0,6,004A07DM0IDW,1d688ec168932,1541555799,7
1,8,009RGHI3G9A3,f05ab0de907e2,1541570940,2
2,10,00Y1Z24X8084,26b6d294d66e7,1541651823,2
3,15,01V3WDTDM5CU,07628a0f5be0b,1541575643,5
4,61,02AOAVF9PVYH,4a01c3afbc224,1541681278,46


In [16]:
na_clickout.tail(3)

,index,user_id,session_id,timestamp,step
253570,3782327,ZYMVSZ5A3KQI,8cd16c29b733b,1541695019,2
253571,3782333,ZZ39YE45SZIE,f5db4092ec9fc,1541551945,6
253572,3782334,ZZCM39YKI3NR,6226bde1465e7,1541601178,1


In [52]:
test2 = test.drop(na_clickout['index'])
test2.reset_index(inplace = True) #index column : test데이터의 실제 index
test2.head(3)

,index,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,0,004A07DM0IDW,1d688ec168932,1541555614,1,interaction item image,2059240,CO,"Santa Marta, Colombia",mobile,NaN,NaN,NaN
1,1,004A07DM0IDW,1d688ec168932,1541555614,2,interaction item image,2059240,CO,"Santa Marta, Colombia",mobile,NaN,NaN,NaN
2,2,004A07DM0IDW,1d688ec168932,1541555696,3,clickout item,1050068,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...


In [0]:
answer = test2[test2.action_type == 'clickout item'].groupby(['user_id', 'session_id']).last().reset_index(drop = True)['index']

In [0]:
test2['answer'] = False

In [0]:
test2.loc[test2['index'].isin(answer.values), 'answer'] = True

In [0]:
test2.drop('index', axis = 1, inplace = True)

In [0]:
test2.to_csv(path + 'test2.csv', index = False)